# Byte Pair Encoding
It was originally created as a data compression algorithm.
Later it was repurposed for subword tokenization which is a very useful method for handing rare words and to manage Out of Vocabulary (OOV) words.

Before jumping on the code, you must read about the Byte Pair Encodin in order to get an idea about it.

Steps involved in BPE:


*   Spilt the words in corpus into characters after appending end word token '</w>'
*   Initialize the vocabulary with unique characters in the corpus
*   Compute the frequency of a pair of characters or character sequences in corpus
*   Merge the most frequent pair in corpus
*   Save the best pair to the vocabulary
*   Repeat 3rd to 5th steps for a certain number of iterations

First read these articles:
* https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/
* https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10 (Well curated article on BPE)
* https://iq.opengenus.org/byte-pair-encoding/ (Must read for better visualization)
*  https://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture12-subwords.pdf (Page no: 18-24_ --> Byte Pair Encoding)
* http://ethen8181.github.io/machine-learning/deep_learning/subword/bpe.html




In [1]:
# import necessary libraries
import re
import string
from collections import Counter, defaultdict
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from tqdm import tqdm_notebook
from textblob import TextBlob
from textblob import Word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# Text corpus
corpus = '''Tropical rainforests are tropical, moist forests of semi-deciduous varieties distributed across nine West African countries. Institute for Sea Research conducted a temperature record dating back to 700,000 years ago.[2] Several conservation and development demographic settings are such that the most loss of rain forests has occurred in countries of higher population growth. Lack of dependable data and survey information in some countries has made the account of areas of unbroken forest and/or under land use change and their relation to economic indicators difficult to ascertain. Hence, the amount and rate of deforestation in Africa are less known than other regions of tropics.
The term deforestation refers to the complete obstruction of forest canopy cover for means of agriculture, plantations, cattle-ranching, and other non-forest fields. Other forest use changes for example are forest disintegration (changing the spatial continuity and creating a mosaic of forest blocks and other land cover types), and dreadful conditions (selective logging of woody species for profitable purposes that affects the forest subfloor and the biodiversity).[2] The general meaning to the term deforestation is linked not only to the value system but the type of measurement designed to assess it. Thus, the same interpretations of deforestation cause noticeable changes in the estimate of forests cleared.
One reason for forest depletion is to grow cash crops. Nine West African countries depend on cash crop exports. Products like gum, copal, rubber, cola nuts, and palm oil provide rather steady income revenue for the West African countries. Land use change spoils entire habitats with the forests. Converting forests into timber is another cause of deforestation. Over decades, the primary forest product was commercial timber. Urbanized countries account for a great percentage of the world's wood consumption, that increased greatly between 1950 and 1980. Simultaneously, preservation measures were reinforced to protect European and American forests.[2] Economic growth and growing environmental protection in industrialized European countries made request for tropical hardwood become strong in West Africa. In the first half of the 1980s, an annual forest loss of 7,200 km2 (2,800 sq mi) was note down along the Gulf of Guinea, a figure equivalent to 4-5 per cent of the total remaining rain forest area.[2] By 1985, 72% of West Africa's rainforests had been transformed into fallow lands and an additional 9% had been opened up by timber exploitation.[2]
Tropical timber became a viable choice to European wood following World War II, as trade with East European countries stop and timber noticeably became sparse in western and southern Europe. Despite efforts to promote lesser known timber species use, the market continued to focus on part of the usable timber obtainable. West Africa was prone to selective harvesting practices; while conservationists blamed the timber industry and the farmers for felling trees, others believe rain forest destruction is connected to the problem of fuel wood.[2] The contribution of fuel wood consumption to tree stock decline in Africa is believed to be significant. It is generally believed that firewood provides 75% of the energy used in sub-Sahara Africa.[2] With the high demand, the consumption of wood for fuel exceeds the renewal of forest cover.
African Pygmies living in the Dzanga-Sangha Special Reserve
The rain forests which remain in West Africa now merely are how they were hardly 30 years ago. In Guinea, Liberia and the Ivory Coast, there is almost no primary forest cover left unscathed; in Ghana the situation is much worse, and nearly all the rain forest are cut down. Guinea-Bissau loses 200 to 350 km2 (77 to 135 sq mi) of forest yearly, Senegal 500 km2 (190 sq mi) of wooded savanna, and Nigeria 6,000,050,000 of both. Liberia exploits 800 km2 (310 sq mi) of forests each year. Extrapolating from present rates of loss, botanist Peter Raven pictures that the majority of the world's moderate and smaller rain forests (such as in Africa) could be ruined in forty years. Tropical Africa is about 18% of the world total covering 20 million km2 (7.7 million sq mi) of land in West and Central Africa.[2] The region has been facing deforestation in various degrees of intensity throughout the recent decades. The actual rate of deforestation varies from one country to another and accurate data does not exist yet. Recent estimates show that the annual pace of deforestation in the region can vary from 150 km2 (58 sq mi) in Gabon to 2,900 km2 (1,100 sq mi) in Cote d'Ivoire. Remaining tropical forest still cover major areas in Central Africa but are abridged by patches in West Africa.
The African Timber Organization member countries (ATO) eventually recognized the cooperation between rural people and their forest environment. Customary law gives residents the right to use trees for firewood, fell trees for construction, and collect of forest products and rights for hunting or fishing and grazing or clearing of forests for maintenance agriculture. Other areas are called "protected forests", which means that uncontrolled clearings and unauthorized logging are forbidden. After World War II, commercial exploitation increased until no West African forestry department was able of making the law. By comparison with rain forests in other places of the world in 1973, Africa showed the greatest infringement though in total volume means, African timber production accounted just one third compared to that of Asia.[2] The difference was due to the variety of trees in Africa forests and the demand for specific wood types in Europe.
Forestry regulations in east Africa were first applied by colonial governments, but they were not strict enough to fill forest exploitation. It wasn't until the 1970s that the inadequate performance of forest regulations was recognized. The Tropical Forestry Action Plan was conceived in 1987 by the World Resources Institute in cooperation with the Food and Agriculture Organization, the United Nations Development Program (UNDP) and the World Bank with hopes of halting tropical forest destruction.[2] In its bid to stress forest conservation and development, the World Bank provided $111,103 million in building countries, especially in Africa, to help in developing long range forest conservation and management programs meant for ending deforestation.'''

corpus

'Tropical rainforests are tropical, moist forests of semi-deciduous varieties distributed across nine West African countries. Institute for Sea Research conducted a temperature record dating back to 700,000 years ago.[2] Several conservation and development demographic settings are such that the most loss of rain forests has occurred in countries of higher population growth. Lack of dependable data and survey information in some countries has made the account of areas of unbroken forest and/or under land use change and their relation to economic indicators difficult to ascertain. Hence, the amount and rate of deforestation in Africa are less known than other regions of tropics.\nThe term deforestation refers to the complete obstruction of forest canopy cover for means of agriculture, plantations, cattle-ranching, and other non-forest fields. Other forest use changes for example are forest disintegration (changing the spatial continuity and creating a mosaic of forest blocks and other l

## Text preprocessing

In [3]:
# lower the sentences
corpus = "".join([word.lower() for word in corpus if word not in string.punctuation])
# remove all digits from the text
pattern = r'[0-9]'
corpus = "".join([re.sub(pattern, '', word) for word in corpus])
corpus = "".join(corpus.split('\n'))
# remove stopwords
stop_words = set(stopwords.words('english'))
corpus = ' '.join([word for word in corpus.split() if word not in stop_words])
corpus

'tropical rainforests tropical moist forests semideciduous varieties distributed across nine west african countries institute sea research conducted temperature record dating back years ago several conservation development demographic settings loss rain forests occurred countries higher population growth lack dependable data survey information countries made account areas unbroken forest andor land use change relation economic indicators difficult ascertain hence amount rate deforestation africa less known regions tropicsthe term deforestation refers complete obstruction forest canopy cover means agriculture plantations cattleranching nonforest fields forest use changes example forest disintegration changing spatial continuity creating mosaic forest blocks land cover types dreadful conditions selective logging woody species profitable purposes affects forest subfloor biodiversity general meaning term deforestation linked value system type measurement designed assess thus interpretation

In [4]:
# word tokens
tokens = word_tokenize(corpus)
corpus = corpus.split()
print(tokens)
print(corpus)
if(tokens == corpus):
  print('True')


['tropical', 'rainforests', 'tropical', 'moist', 'forests', 'semideciduous', 'varieties', 'distributed', 'across', 'nine', 'west', 'african', 'countries', 'institute', 'sea', 'research', 'conducted', 'temperature', 'record', 'dating', 'back', 'years', 'ago', 'several', 'conservation', 'development', 'demographic', 'settings', 'loss', 'rain', 'forests', 'occurred', 'countries', 'higher', 'population', 'growth', 'lack', 'dependable', 'data', 'survey', 'information', 'countries', 'made', 'account', 'areas', 'unbroken', 'forest', 'andor', 'land', 'use', 'change', 'relation', 'economic', 'indicators', 'difficult', 'ascertain', 'hence', 'amount', 'rate', 'deforestation', 'africa', 'less', 'known', 'regions', 'tropicsthe', 'term', 'deforestation', 'refers', 'complete', 'obstruction', 'forest', 'canopy', 'cover', 'means', 'agriculture', 'plantations', 'cattleranching', 'nonforest', 'fields', 'forest', 'use', 'changes', 'example', 'forest', 'disintegration', 'changing', 'spatial', 'continuity',

In [5]:
# Lemmatization
wordnet_lemmatizer = WordNetLemmatizer() 
corpus = [wordnet_lemmatizer.lemmatize(word) for word in corpus]

In [6]:
print(corpus)

['tropical', 'rainforest', 'tropical', 'moist', 'forest', 'semideciduous', 'variety', 'distributed', 'across', 'nine', 'west', 'african', 'country', 'institute', 'sea', 'research', 'conducted', 'temperature', 'record', 'dating', 'back', 'year', 'ago', 'several', 'conservation', 'development', 'demographic', 'setting', 'loss', 'rain', 'forest', 'occurred', 'country', 'higher', 'population', 'growth', 'lack', 'dependable', 'data', 'survey', 'information', 'country', 'made', 'account', 'area', 'unbroken', 'forest', 'andor', 'land', 'use', 'change', 'relation', 'economic', 'indicator', 'difficult', 'ascertain', 'hence', 'amount', 'rate', 'deforestation', 'africa', 'le', 'known', 'region', 'tropicsthe', 'term', 'deforestation', 'refers', 'complete', 'obstruction', 'forest', 'canopy', 'cover', 'mean', 'agriculture', 'plantation', 'cattleranching', 'nonforest', 'field', 'forest', 'use', 'change', 'example', 'forest', 'disintegration', 'changing', 'spatial', 'continuity', 'creating', 'mosaic',

In [7]:
corpus = " ".join(corpus)
print(corpus)

tropical rainforest tropical moist forest semideciduous variety distributed across nine west african country institute sea research conducted temperature record dating back year ago several conservation development demographic setting loss rain forest occurred country higher population growth lack dependable data survey information country made account area unbroken forest andor land use change relation economic indicator difficult ascertain hence amount rate deforestation africa le known region tropicsthe term deforestation refers complete obstruction forest canopy cover mean agriculture plantation cattleranching nonforest field forest use change example forest disintegration changing spatial continuity creating mosaic forest block land cover type dreadful condition selective logging woody specie profitable purpose affect forest subfloor biodiversity general meaning term deforestation linked value system type measurement designed ass thus interpretation deforestation cause noticeable 

## Step 1: Build vocab from text corpus

In [8]:
def build_vocab(corpus: str) -> dict:
  # Separate each character in every word in the corpus by a space and add mark end os token
  tokens = [" ".join(word) + " </w>" for word in corpus.split()]
  # Count frequency of tokens in corpus
  vocab = Counter(tokens)
  return vocab

In [9]:
build_vocab(corpus).items()

dict_items([('t r o p i c a l </w>', 7), ('r a i n f o r e s t </w>', 2), ('m o i s t </w>', 1), ('f o r e s t </w>', 36), ('s e m i d e c i d u o u s </w>', 1), ('v a r i e t y </w>', 2), ('d i s t r i b u t e d </w>', 1), ('a c r o s s </w>', 1), ('n i n e </w>', 2), ('w e s t </w>', 10), ('a f r i c a n </w>', 6), ('c o u n t r y </w>', 11), ('i n s t i t u t e </w>', 2), ('s e a </w>', 1), ('r e s e a r c h </w>', 1), ('c o n d u c t e d </w>', 1), ('t e m p e r a t u r e </w>', 1), ('r e c o r d </w>', 1), ('d a t i n g </w>', 1), ('b a c k </w>', 1), ('y e a r </w>', 4), ('a g o </w>', 2), ('s e v e r a l </w>', 1), ('c o n s e r v a t i o n </w>', 3), ('d e v e l o p m e n t </w>', 3), ('d e m o g r a p h i c </w>', 1), ('s e t t i n g </w>', 1), ('l o s s </w>', 3), ('r a i n </w>', 7), ('o c c u r r e d </w>', 1), ('h i g h e r </w>', 1), ('p o p u l a t i o n </w>', 1), ('g r o w t h </w>', 2), ('l a c k </w>', 1), ('d e p e n d a b l e </w>', 1), ('d a t a </w>', 2), ('s u r

## Get counts of pairs of consecutive sequences

In [10]:
def get_stats(vocab: dict) ->  str:
  pairs = defaultdict(int)
  for word, frequency in vocab.items():
    symbols = word.split()
    # Counting up frequencies of pairs
    for i in range(len(symbols)-1):
      pairs[symbols[i], symbols[i+1]] += frequency
  return pairs

## Step 3: Merge all occurences of the most frequent pair

In [11]:
def merge_vocab(pair: tuple, v_in: dict) -> dict:
  v_out = {}

  # re.escape
  # ensures the characters of our input pair will be handled as it is and
  # not get mistreated as special characters in the regular expression.
  bigram = re.escape(' '.join(pair))
  p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
  for word in v_in:
    w_out = p.sub(''.join(pair), word)   # For eg: 'a b' gets replaced by 'ab' in 'a b b c a d a b'
    v_out[w_out] = v_in[word]
  return v_out

## Run all the functions

In [12]:
# step 1
vocab = build_vocab(corpus)
# hyperparameter 
# number of iterations of merges we want in our vocabulary
num_merges = 300
# Step 2
for i in range(num_merges):
  pairs = get_stats(vocab)
  if not pairs:
    break
  # Step 3
  best = max(pairs, key=pairs.get)
  vocab = merge_vocab(best, vocab)

In [13]:
# Final vacabulary
print(vocab)

{'tropical</w>': 7, 'rainforest</w>': 2, 'mo ist</w>': 1, 'forest</w>': 36, 'se mi de ci du ous</w>': 1, 'variety</w>': 2, 'di stri bu ted</w>': 1, 'ac ro ss</w>': 1, 'nine</w>': 2, 'west</w>': 10, 'african</w>': 6, 'country</w>': 11, 'institute</w>': 2, 'se a</w>': 1, 'res ear ch</w>': 1, 'con duc ted</w>': 1, 't e mp er a ture</w>': 1, 'reco rd</w>': 1, 'd ating</w>': 1, 'b ack</w>': 1, 'year</w>': 4, 'ago</w>': 2, 'se ver al</w>': 1, 'conservation</w>': 3, 'development</w>': 3, 'de mo gra p h ic</w>': 1, 'se t ting</w>': 1, 'loss</w>': 3, 'rain</w>': 7, 'oc cu r re d</w>': 1, 'hi gh er</w>': 1, 'p op ulation</w>': 1, 'growth</w>': 2, 'l ack</w>': 1, 'depen d able</w>': 1, 'data</w>': 2, 'su r ve y</w>': 1, 'in form ation</w>': 1, 'made</w>': 2, 'account</w>': 2, 'area</w>': 4, 'un b ro k en</w>': 1, 'an d or</w>': 1, 'land</w>': 5, 'use</w>': 5, 'change</w>': 4, 're l ation</w>': 1, 'economic</w>': 2, 'in d ic at or</w>': 1, 'diff ic ul t</w>': 1, 'as c er tain </w>': 1, 'h en ce</w

Here, ends the Byte Pair Encoding algorithm!
I hope you understood the main concepts behind the algorithm.

## Visualize the output of various steps below

In [14]:
print(vocab)
print(len(vocab))

{'tropical</w>': 7, 'rainforest</w>': 2, 'mo ist</w>': 1, 'forest</w>': 36, 'se mi de ci du ous</w>': 1, 'variety</w>': 2, 'di stri bu ted</w>': 1, 'ac ro ss</w>': 1, 'nine</w>': 2, 'west</w>': 10, 'african</w>': 6, 'country</w>': 11, 'institute</w>': 2, 'se a</w>': 1, 'res ear ch</w>': 1, 'con duc ted</w>': 1, 't e mp er a ture</w>': 1, 'reco rd</w>': 1, 'd ating</w>': 1, 'b ack</w>': 1, 'year</w>': 4, 'ago</w>': 2, 'se ver al</w>': 1, 'conservation</w>': 3, 'development</w>': 3, 'de mo gra p h ic</w>': 1, 'se t ting</w>': 1, 'loss</w>': 3, 'rain</w>': 7, 'oc cu r re d</w>': 1, 'hi gh er</w>': 1, 'p op ulation</w>': 1, 'growth</w>': 2, 'l ack</w>': 1, 'depen d able</w>': 1, 'data</w>': 2, 'su r ve y</w>': 1, 'in form ation</w>': 1, 'made</w>': 2, 'account</w>': 2, 'area</w>': 4, 'un b ro k en</w>': 1, 'an d or</w>': 1, 'land</w>': 5, 'use</w>': 5, 'change</w>': 4, 're l ation</w>': 1, 'economic</w>': 2, 'in d ic at or</w>': 1, 'diff ic ul t</w>': 1, 'as c er tain </w>': 1, 'h en ce</w

In [15]:
pairs = get_stats(vocab)
print(pairs)

defaultdict(<class 'int'>, {('mo', 'ist</w>'): 1, ('se', 'mi'): 1, ('mi', 'de'): 1, ('de', 'ci'): 1, ('ci', 'du'): 1, ('du', 'ous</w>'): 1, ('di', 'stri'): 1, ('stri', 'bu'): 1, ('bu', 'ted</w>'): 1, ('ac', 'ro'): 1, ('ro', 'ss</w>'): 1, ('se', 'a</w>'): 1, ('res', 'ear'): 1, ('ear', 'ch</w>'): 1, ('con', 'duc'): 1, ('duc', 'ted</w>'): 1, ('t', 'e'): 1, ('e', 'mp'): 1, ('mp', 'er'): 1, ('er', 'a'): 1, ('a', 'ture</w>'): 1, ('reco', 'rd</w>'): 1, ('d', 'ating</w>'): 1, ('b', 'ack</w>'): 1, ('se', 'ver'): 1, ('ver', 'al</w>'): 1, ('de', 'mo'): 1, ('mo', 'gra'): 1, ('gra', 'p'): 1, ('p', 'h'): 1, ('h', 'ic</w>'): 1, ('se', 't'): 1, ('t', 'ting</w>'): 1, ('oc', 'cu'): 1, ('cu', 'r'): 1, ('r', 're'): 1, ('re', 'd</w>'): 1, ('hi', 'gh'): 1, ('gh', 'er</w>'): 1, ('p', 'op'): 1, ('op', 'ulation</w>'): 1, ('l', 'ack</w>'): 1, ('depen', 'd'): 1, ('d', 'able</w>'): 1, ('su', 'r'): 1, ('r', 've'): 1, ('ve', 'y</w>'): 1, ('in', 'form'): 1, ('form', 'ation</w>'): 1, ('un', 'b'): 1, ('b', 'ro'): 1, (

In [16]:
best = max(pairs, key=pairs.get)
print(best)
print(pairs[best])

('k', 'no')
2
